# Grounding in Logic Tensor Networks (LTN)

## Real Logic

The semantics of LTN differs from the standard abstract semantics of First-order Logic (FOL) in that domains are interpreted concretely by tensors in the Real field.
To emphasize the fact that LTN interprets symbols which are grounded on real-valued features, we use the term *grounding*, denoted by $\mathcal{G}$, instead of interpretation.
$\mathcal{G}$ associates a tensor of real numbers to any term of the language, and a real number in the interval $[0,1]$ to any formula $\phi$.
In the rest of the tutorials, we commonly use "tensor" to designate "tensor in the Real field".

The language consists of a non-logical part (the signature) and logical connectives and quantifiers.
* **constants** denote individuals from some space of tensors $\bigcup\limits_{n_1 \dots n_d \in \mathbb{N}^*} \mathbb{R}^{n_1 \times \dots \times n_d}$ (tensor of any rank). The individual can be pre-defined (data point) or learnable (embedding).
* **variables** denote sequences of individuals.
* **functions** can be any mathematical function either pre-defined or learnable. Examples of functions can be distance functions, regressors, etc. Functions can be defined using any operations in PyTorch. They can be linear functions, Deep Neural Networks, and so forth.
* **predicates** are represented as mathematical functions that map from some n-ary domain of individuals to a real number in $[0,1]$ that can be interpreted as a truth degree. Examples of predicates can be similarity measures, classifiers, etc.
* **connectives** -- not, and, or, implies -- are modeled using fuzzy semantics.
* **quantifiers** are defined using aggregators.

This tutorial explains how to ground constants, variables, functions and predicates.

In [1]:
import ltn
import torch
import numpy as np

## Constants

LTN constants are grounded into real tensors. Each constant $c$ is mapped to a point in $\mathcal{G}(c) \in \bigcup\limits_{n_1 \dots n_d \in \mathbb{N}^*} \mathbb{R}^{n_1 \times \dots \times n_d}$. Notice that the objects in the domain may be tensors of any rank. A tensor of rank $0$ corresponds to a scalar, a tensor of rank $1$ to a vector, a tensor of rank $2$ to a matrix and so forth, in the usual way.
Here we define $\mathcal{G}(c_1)=(2.1,3)$ and $\mathcal{G}(c_2)=\begin{pmatrix}
4.2 & 3 & 2.5\\
4 & -1.3 & 1.8
\end{pmatrix}$.

In [2]:
c1 = ltn.constant([2.1, 3])
c2 = ltn.constant([[4.2, 3, 2.5], [4, -1.3, 1.8]])

Note that a constant can be set as learnable by using the optional argument `trainable=True`. This is useful to learn embeddings for some individuals.
The features of the tensor will be considered as trainable parameters (learning in LTN is explained in a further notebook).

In [3]:
c3 = ltn.constant([0.,0.], trainable=True)

LTN constants are implemented using PyTorch tensors, with some added dynamic attributes (i.e., `free_variables`, which is explained later). If the parameter `trainable` of the LTN constant is set to `True`, then
the `requires_grad` parameter of the PyTorch tensor representing the LTN constant will be set to `True`. Since the constants are PyTorch tensors,
it is possible to query the value of the constants easily.

In [4]:
print(c1)
print(c1.numpy())
print(c3)
# here, we have to perform a detach before calling numpy(), because the tensor has requires_grad=True
print(c3.detach().numpy())

tensor([2.1000, 3.0000])
[2.1 3. ]
tensor([0., 0.], requires_grad=True)
[0. 0.]


## Predicates

LTN predicates are grounded in mappings that assign a value between zero and one to some n-ary space of input values. Predicates in LTN can be neural networks or any other function that achieves such a mapping.

There are different ways to construct a predicate in LTN. The constructor `ltn.Predicate(model, layers_size, lambda_func)` permits three types of construction:
- if the `model` parameter is not `None`, the predicate is constructed by using the `torch.nn.Module` model instance that has been given as input;
- if the `model` parameter is `None` and the `layers_size` parameter is not `None`, the predicate is constructed using a fully-connected MLP as the predicate model. A `torch.nn.Sequential` model is constructed
using the size of the layers provided in the `layers_size` parameter, which has to be a tuple of integers. Note that the tuple must contain the size of the input and output layer too. An ELU activation is used on the hidden layers, and a sigmoid
activation is used on the final layer;
- if the `model` parameter is `None`, the `layers_size` parameter is `None`, and the `lambda_func` parameter is not `None`, the predicate is constructed by using the lambda function given in input. The construction by using
the lambda function is suggested for small mathematical operations with **no weight tracking** (non-trainable function).

The following defines a predicate $P_1$ using the lambda_func parameter, a predicate $P_2$ using the `model` parameter, and a predicate
$P_3$ using the `layers_size` parameter.

In [9]:
mu = ltn.constant([2., 3.])
P1 = ltn.Predicate(lambda_func=lambda x: torch.exp(-torch.norm(x[0] - mu, dim=1)))

class ModelP2(torch.nn.Module):
    """For more info on how to use torch.nn.Module:
    https://pytorch.org/docs/stable/generated/torch.nn.Module.html"""
    def __init__(self):
        super(ModelP2, self).__init__()
        self.elu = torch.nn.ELU()
        self.sigmoid = torch.nn.Sigmoid()
        self.dense1 = torch.nn.Linear(2, 5)
        self.dense2 = torch.nn.Linear(5, 1) # returns one value in [0,1]

    def forward(self, x):
        x = x[0]  # here, we have to access to x[0] because the input is always contained in a list due to LTN pre-processing
        x = self.elu(self.dense1(x))
        return self.sigmoid(self.dense2(x))

modelP2 = ModelP2()
P2 = ltn.Predicate(model=modelP2)

P3 = ltn.Predicate(layers_size=(2, 5, 1))

One can easily query predicates using LTN constants and LTN variables (see further in this notebook to query using variables).

In [10]:
c1 = ltn.constant([2.1, 3])
c2 = ltn.constant([4.5, 0.8])
c3 = ltn.constant([3.0, 4.8])
print(P1(c1))
print(P1(c2))
print(P2(c3))
print(P3(c3))

tensor(0.9048)
tensor(0.0358)
tensor(0.4690, grad_fn=<ViewBackward>)
tensor(0.4452, grad_fn=<ViewBackward>)


NOTE:
- If an LTN predicate (or an LTN function) takes several inputs, e.g. $P_4(x_1,x_2)$, the arguments must be passed as a list.
- LTN converts inputs such that there is a "batch" dimension on the first dim. Therefore, most operations should work with `dim=1`.

In [13]:
class ModelP4(torch.nn.Module):
    def __init__(self):
        super(ModelP4, self).__init__()
        self.elu = torch.nn.ELU()
        self.sigmoid = torch.nn.Sigmoid()
        self.dense1 = torch.nn.Linear(4, 5)
        self.dense2 = torch.nn.Linear(5, 1) # returns one value in [0,1]

    def forward(self, x):
        x = torch.cat(x, dim=1)
        x = self.elu(self.dense1(x))
        return self.sigmoid(self.dense2(x))

P4 = ltn.Predicate(ModelP4())
c1 = ltn.constant([2.1, 3])
c2 = ltn.constant([4.5, 0.8])
print(P4([c1, c2])) # multiple arguments are passed as a list

tensor(0.5946, grad_fn=<ViewBackward>)


One may define trainable or non-trainable 0-ary predicates (propositional variables) using `ltn.propositional_variable`.
They are grounded as a mathematical constant in $[0,1]$. Notice that if you decide to set the propositional variable as
trainable, you should constraint its value in the range $[0,1]$ during learning. To do that, it is possible to use
`torch.clamp()`. To understand how this works, it is possible to go through the `propositional_variables.py` example included
in the repository. Note that LTNtorch will not constraint the value of a trainable propositional variable automatically.
This is why a warning is printed when a trainable propositional variable is declared.

In [2]:
# Declaring a trainable 0-ary predicate with initial truth value 0.3
A = ltn.propositional_variable(0.3, trainable=True)

/Users/ciomi/PycharmProjects/LTNtorch/ltn/core.py:99: UserWarning: Attention! You have defined a trainable LTN propositional variable. Therefore, you should constraint its value in [0., 1.] during learning. LTNtorch does not do that automatically. It is possible to use torch.clamp to do that. See the propositional variables example to understand how this works.
  warnings.warn("Attention! You have defined a trainable LTN propositional variable. Therefore, you should "


For more details and useful ways to create predicates (incl. how to integrate multiclass classifiers as binary predicates) see the complementary notebook.

## Functions

LTN functions are grounded as any mathematical function that maps $n$ individuals to one individual in the tensor domains.

There are different ways to construct a function in LTN. The constructor `ltn.Function(model, layers_size, lambda_func)` permits three types of construction:
- if the `model` parameter is not `None`, the function is constructed by using the `torch.nn.Module` model instance that has been given as input;
- if the `model` parameter is `None` and the `layers_size` parameter is not `None`, the function is constructed using a fully-connected MLP as the function model. A `torch.nn.Sequential` model is constructed
using the size of the layers provided in the `layers_size` parameter, which has to be a tuple of integers. Note that the tuple must contain the size of the input and output layer too. An ELU activation is used on the hidden layers,
and a linear activation is used on the final layer;
- if the `model` parameter is `None`, the `layers_size` parameter is `None`, and the `lambda_func` parameter is not `None`, the function is constructed by using the lambda function given in input. The construction by using
a lambda function is suggested for small mathematical operations with **no weight tracking** (non-trainable function).

The following defines a function $f_1$ using the lambda_func parameter, a function $f_2$ using the `model` parameter, and a function
$f_3$ using the `layers_size` parameter.

In [14]:
f1 = ltn.Function(lambda_func=lambda args: args[0] - args[1])

class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = torch.nn.Linear(2, 10)
        self.dense2 = torch.nn.Linear(10, 5)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = x[0]
        x = self.relu(self.dense1(x))
        return self.dense2(x)

model_f2 = MyModel()
f2 = ltn.Function(model=model_f2)

f3 = ltn.Function(layers_size=(2, 10, 5))

One can easily compute functions using LTN constants and LTN variables as inputs (see further in this notebook to query using variables).

In [15]:
c1 = ltn.constant([2.1, 3])
c2 = ltn.constant([4.5, 0.8])
print(f1([c1, c2])) # multiple arguments are passed as a list
print(f2(c1))
print(f3(c1))

tensor([-2.4000,  2.2000])
tensor([-0.0324,  0.5578,  0.4427, -0.2395,  0.4639], grad_fn=<ViewBackward>)
tensor([ 1.0815, -0.8840,  0.6184, -0.2315, -0.5752], grad_fn=<ViewBackward>)


## Variables

LTN variables are sequences of individuals/constants from a domain. Variables are useful to write quantified statements, e.g. $\forall x\ P(x)$. Notice that a variable is a sequence and not a set, namely, the same value can occur twice in the sequence.

The following defines two variables, $x$ and $y$, with respectively 10 and 5 individuals sampled from normal distributions in $\mathbb{R}^2$.
In LTN, variables need to be labelled (see the arguments `'x'` and `'y'` below).

In [16]:
x = ltn.variable('x', np.random.normal(0., 1., (10, 2)))
y = ltn.variable('y', np.random.normal(0., 4., (5, 2)))

Evaluating a term/predicate on one variable of $n$ individuals yields $n$ output values, where the $i$-th output value corresponds to the term calculated with the $i$-th individual.

Similarly, evaluating a term/predicate on $k$ variables $(x_1,\dots,x_k)$, with respectively $n_1,\dots,n_k$ individuals each, yields a result with $n_1 \times \dots \times n_k$ values. The result is organized in a tensor where the first $k$ dimensions can be indexed to retrieve the outcome(s) that correspond to each variable. The tensor has a dynamically added attribute `free_variables` that tells which axis corresponds to which variable (using the label of the variable).

In [19]:
# Notice that the outcome is a 2-dimensional tensor where each cell
# represents the satisfiability of P4 evaluated with each individual in x and in y.
P4 = ltn.Predicate(ModelP4())
res1 = P4([x, y])
print(res1.shape)
print(res1.free_variables) # dynamically added attribute; tells that axis 0 corresponds to x and axis 1 to y
print(res1[2, 0]) # gives the result computed with the 3rd individual in x and the 1st individual in y

torch.Size([10, 5])
['x', 'y']
tensor(0.4975, grad_fn=<SelectBackward>)


In [20]:
# Notice that the last axe(s) correspond to the dimensions of the outcome;
# here, f2 projects to outcomes in R^2, so the outcome has one additional axis of dimension 2.
# the output tensor has shape (10, 5, 2) because variable x has 10 individuals, y has 5 individuals, and f1 maps in R^2
res2 = f1([x, y])
print(res2.shape)
print(res2.free_variables)
print(res2[2,0]) # gives the result calculated with the 3rd individual in x and the 1st individual in y

torch.Size([10, 5, 2])
['x', 'y']
tensor([7.5669, 2.0605])


In [23]:
c1 = ltn.constant([2.1, 3])
res3 = P4([c1, y])
print(res3.shape) # Notice that no axis is associated to a constant. The output has shape (5,) because variable y has
# 5 individuals
print(res3.free_variables)
print(res3[0]) # gives the result calculated with c1 and the 1st individual in y

torch.Size([5])
['y']
tensor(0.5619, grad_fn=<SelectBackward>)


### Variables made of trainable constants

A variable can be instantiated using two different types of objects:
- A value (numpy, python list, ...) that will be fed in a `torch.FloatTensor` (the variable refers to a new object).
- A `torch.FloatTensor` instance that will be used directly as the variable (the variable refers to the same object).

The latter is useful when the variable denotes a sequence of trainable constants. In order to better understand this
topic, it is kindly suggested to carefully read the `Smokes-friends-cancer` example, where variables
made of trainable constants are used.

In [25]:
c1 = ltn.constant([2.1, 3], trainable=True)
c2 = ltn.constant([4.5, 0.8], trainable=True)

# PyTorch will keep track of the gradients between c1, c2 and x.
# Read tutorial 3 for more details.
x = ltn.variable('x', torch.stack([c1,c2]))
res = P2(x)
print(res)

tensor([0.5075, 0.6637], grad_fn=<ViewBackward>)
